In [54]:
from draughts import Draughts
import random
import sys
import numpy as np
import time

### Define heuristic players for MCTS

* **RandomPlayer** – selects moves entirely at random using the `random.choice()` function.
* **RandomPlayerCaptureIfPossible** – behaves like `RandomPlayer`, but prioritizes capturing opponent pieces when possible.


In [55]:
class RandomPlayer:
    def __init__(self, name="Random Player"):
        self.name = name

    def move(self, board, player_id, valid_moves):
        a = random.choice(valid_moves)
        return a

class RandomPlayerCaptureIfPossible:
    def __init__(self, name="Random Player Capturer"):
        self.name = name   

    def move(self, board, player_id, valid_moves):
        capture_moves = [move for move in valid_moves if move['captures']]
        if len(capture_moves) > 0:
            return random.choice(capture_moves)
        return random.choice(valid_moves)


player1 = RandomPlayer()
player2 = RandomPlayerCaptureIfPossible()
# Draughts(player1, player1).play()
results = {player1.name: 0, player2.name: 0}
for i in range(10):
    game = Draughts(player1, player2)
    result = game.play(verbose=False)
    if result == -1:
        results[player1.name] += 1
    else:
        results[player2.name] += 1
    game = Draughts(player2, player1)
    result = game.play(verbose=False)
    if result == -1:
        results[player2.name] += 1
    else:
        results[player1.name] += 1
print(results)
    


{'Random Player': 18, 'Random Player Capturer': 2}


It seems that player who plays all at random is better than player who seeks for capture.

### Implementation of **Monte Carlo Tree Search**

The algorithm follows four main steps:

* **Selection** – traverse through *n* random possible moves (all starting from the default state).
* **Expansion** – create *n* instances of the game (if no winning move is found).
* **Playout** – simulate the games using `game.play()` with the given `heuristics_player`, calculating the likelihood of winning.
* **Backpropagation** – choose the move with the highest likelihood of winning (or select the immediate winning move, if available).


In [63]:
class MonteCarloTreeSearchPlayer:
    def __init__(self, name="MCTS Player", heuristics_player=RandomPlayer(), n=None, simulations=100):
        self.name = name
        self.heuristics_player = heuristics_player
        self.n = n
        self.simulations = simulations

    def calculate_likelihood(self, move, board, player_id):
        wins = 0
        game = Draughts(self.heuristics_player, self.heuristics_player)
        for _ in range(self.simulations):
            game.set_board(board, player_id)
            game.make_move(move)
            
            # if game is over, curent player wins so likelihood is 100%
            if game.check_for_end():
                return 1
            
            result = game.play(verbose=False)
            if result == player_id:
                wins += 1
        return wins / self.simulations

    def move(self, board, player_id, valid_moves):
        
        if self.n is None or self.n <= len(valid_moves):
            compute_moves = valid_moves
        else:
            compute_moves = random.sample(valid_moves, self.n)
            
        likelihoods = np.array([self.calculate_likelihood(move, board, player_id) for move in compute_moves])
        max_index = np.argmax(likelihoods)
        best_move = compute_moves[max_index]
        return best_move



In [64]:
Draughts(RandomPlayer(), MonteCarloTreeSearchPlayer()).play()

    0   1   2   3   4   5   6   7 
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
0 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
1 │ ○ │   │ ○ │   │ ○ │   │ ○ │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
2 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
3 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
4 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
5 │ ● │   │ ● │   │ ● │   │ ● │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
6 │   │ ● │   │ ● │   │ ● │   │ ● │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
7 │ ● │   │ ● │   │ ● │   │ ● │   │


    0   1   2   3   4   5   6   7 
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
0 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
1 │ ○ │   │ ○ │   │ ○ │   │ ○ │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
2 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
3 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
4 │   │ ● │   │   │   │   │ 

1

In [ ]:
player1 = MonteCarloTreeSearchPlayer()
player2 = RandomPlayer()
results = {player1.name: 0, player2.name: 0}
n = 2 
print(f"Starting {2* n} games, comparing {player1.name} vs {player2.name}...")
for i in range(n):
    print(f"Game {i+1}/{2*n}: {player1.name} (player 1) vs {player2.name} (player 2)...", end=" ")
    game = Draughts(player1, player2)
    result = game.play(verbose=False)
    if result == -1:
        results[player1.name] += 1
        print(f"{player1.name} wins!")
    else:
        results[player2.name] += 1
        print(f"{player2.name} wins!")
for i in range(10):
    print(f"Game {i+n+1}/{2*n}: {player2.name} (player 1) vs {player1.name} (player 2)...", end=" ")
    game = Draughts(player2, player1)
    result = game.play(verbose=False)
    if result == -1:
        results[player2.name] += 1
        print(f"{player2.name} wins!")
    else:
        results[player1.name] += 1
        print(f"{player1.name} wins!")
print(results)
    

Starting 4 games, comparing MCTS Player vs Random Player...
Game 1/4: MCTS Player (player 1) vs Random Player (player 2)... MCTS Player wins!
Game 2/4: MCTS Player (player 1) vs Random Player (player 2)... MCTS Player wins!
Game 3/4: Random Player (player 1) vs MCTS Player (player 2)... 

KeyboardInterrupt: 

### MCTS player with Ray
implementation for parallel computing with ray

In [57]:
import ray
import os

if ray.is_initialized:
    print("Ray is going to be shut down")
    ray.shutdown()
ray.init(ignore_reinit_error=True)

Ray is going to be shut down


2025-05-11 00:09:25,225	INFO worker.py:1879 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.12.3
Ray version:,2.46.0
Dashboard:,http://127.0.0.1:8265


In [61]:
@ray.remote
def calculate_likelihood(heuristics_player, move, board, player_id, simulations):
    wins = 0
    game = Draughts(heuristics_player, heuristics_player)
    for _ in range(simulations):
        game.set_board(board, player_id)
        game.make_move(move)
        
        # if game is over, curent player wins so likelihood is 100%
        if game.check_for_end():
            return 1
        
        result = game.play(verbose=False)
        if result == player_id:
            wins += 1
    return wins / simulations


class RayMCTSPlayer:
    def __init__(self, name="Ray MCTS Player", heuristics_player=RandomPlayer(), n=None, simulations=100):
        self.name = name
        self.heuristics_player = heuristics_player
        self.n = n
        self.simulations = simulations


    def slice_moves_list(self, compute_moves):
        slices_num = os.cpu_count()
        slices = []
        part_size = len(compute_moves) // slices_num
        remainder = len(compute_moves) % slices_num
        start = 0
        for i in range(slices_num):
            end = start + part_size + (1 if i < remainder else 0)
            slices.append(compute_moves[start:end])
            start = end
        return slices
    
    def move(self, board, player_id, valid_moves):
        
        if self.n is None or self.n <= len(valid_moves):
            compute_moves = valid_moves
        else:
            compute_moves = random.sample(valid_moves, self.n)

        best_move = None
        best_likelihood = 0
        likehoods = [calculate_likelihood.remote(
            self.heuristics_player, 
            move, 
            board, player_id, 
            self.simulations
            )
                      for move in compute_moves]
        likelihoods = ray.get(likehoods)
        max_index = np.argmax(likelihoods)
        best_move = compute_moves[max_index]
        return best_move


In [62]:
Draughts(RayMCTSPlayer(), RayMCTSPlayer()).play()

    0   1   2   3   4   5   6   7 
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
0 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
1 │ ○ │   │ ○ │   │ ○ │   │ ○ │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
2 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
3 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
4 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
5 │ ● │   │ ● │   │ ● │   │ ● │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
6 │   │ ● │   │ ● │   │ ● │   │ ● │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
7 │ ● │   │ ● │   │ ● │   │ ● │   │


    0   1   2   3   4   5   6   7 
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
0 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
1 │ ○ │   │ ○ │   │ ○ │   │ ○ │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
2 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
3 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
4 │   │   │   │   │   │ ● │ 

1

## measering execution time
computing of expected game time xexcution with diffrent `samples` number

In [ ]:
from time_measuring import measure_execution_time, plot_execution_time_distribution

simulations = [10, 20, 100, 200, 300]

times = {"ray": dict(), "no_ray": dict()}

for simulation_no in simulations:
    print(f"Simulation_no calculating {simulation_no}...", end=" ")
    f_ray = Draughts(RayMCTSPlayer(simulations=simulation_no), RandomPlayer()).play
    f_no_ray = Draughts(MonteCarloTreeSearchPlayer(simulations=simulation_no), RandomPlayer()).play

    times["ray"][simulation_no] = measure_execution_time(f_ray)
    print(f"Ray time: {np.mean(times['ray'][simulation_no]) * 1000:.4f} ms ...", end=" ")
    times["no_ray"][simulation_no] = measure_execution_time(f_no_ray)
    print(f"No ray time: {np.mean(times['no_ray'][simulation_no]) * 1000:.4f} ms")

    0   1   2   3   4   5   6   7 
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
0 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
1 │ ○ │   │ ○ │   │ ○ │   │ ○ │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
2 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
3 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
4 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
5 │ ● │   │ ● │   │ ● │   │ ● │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
6 │   │ ● │   │ ● │   │ ● │   │ ● │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
7 │ ● │   │ ● │   │ ● │   │ ● │   │


    0   1   2   3   4   5   6   7 
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
0 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
1 │ ○ │   │ ○ │   │ ○ │   │ ○ │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
2 │   │ ○ │   │ ○ │   │ ○ │   │ ○ │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
3 │   │   │   │   │   │   │   │   │
  ┼───┼───┼───┼───┼───┼───┼───┼───┼
4 │   │   │   │ ● │   │   │ 

KeyboardInterrupt: 

In [ ]:
ray.shutdown()